In [1]:
import glob
import io
import json
import os
import shutil
import subprocess
import sys

import numpy as np
import pandas as pd
from Bio import AlignIO, Entrez, SearchIO, SeqIO
from Bio.Align import MultipleSeqAlignment
from Bio.Align.AlignInfo import SummaryInfo
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from ete3 import Tree
from pynucl.hist_features import hist_features

/opt/miniconda3/envs/moldyn/lib/python3.7/site-packages/Bio/Align/substitution_matrices/__init__.py:21: BiopythonExperimentalWarning: Bio.Align.substitution_matrices is an experimental module which may still undergo significant changes. In particular, the location of this module may change, and the Array class defined in this module may be moved to other existing or new modules in Biopython.
  BiopythonExperimentalWarning)


In [2]:
def dict2tree(tree, d):
    """
    converts tree from classification.json to a ete3 object
    d is
    with open('classification.json') as json_file:
        data = json.load(json_file)
    d=data['tree']
    """
    for k, v in d.items():
        CH = tree.add_child(name=k)
        if isinstance(v, dict):
            dict2tree(CH, v)

In [3]:
def muscle_aln(sequences, options=[], debug=False, sort=True):
    muscle = os.path.join(os.path.dirname(sys.executable), "muscle")
    process = subprocess.Popen(
        [muscle] + options,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
    )
    aln, error = process.communicate(sequences.encode("utf-8"))
    if debug:
        print(sequences)
        print()
        print("Stderr:")
        print(error.decode("utf-8"))
        print("Stdout:")
        print(aln.decode("utf-8"))
    seqFile = io.StringIO()
    seqFile.write(aln.decode("utf-8"))
    seqFile.seek(0)
    sequences_ids = [s.split(" ", 1)[0] for s in sequences.split(">")]
    sequences = list(
        SeqIO.parse(seqFile, "fasta")
    )  # Not in same order, but does it matter?
    if sort:
        sequences.sort(key=lambda x: sequences_ids.index(x.id))  # Yes, it matters
    msa = MultipleSeqAlignment(sequences)
    return msa

In [4]:
def muscle_p2p_aln(msa1, msa2, options=[], debug=False):
    """
    align two alignments
    :return: MultipleSeqAlignment object
    """
    os.makedirs("tmp/")  # create tmp dir to save msa profiles
    try:
        with open("tmp/one.afa", "w") as f:
            f.write(format(msa1, "fasta"))
        with open("tmp/two.afa", "w") as f:
            f.write(format(msa2, "fasta"))

        muscle = os.path.join(os.path.dirname(sys.executable), "muscle")
        process = subprocess.Popen(
            [muscle]
            + options
            + ["-profile", "-in1", "tmp/one.afa", "-in2", "tmp/two.afa"],
            stdin=subprocess.PIPE,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
        )

        aln, error = process.communicate()
        if debug:
            print("Stderr:")
            print(error.decode("utf-8"))
            print("Stdout:")
            print(aln.decode("utf-8"))

        seqFile = io.StringIO()
        seqFile.write(aln.decode("utf-8"))
        seqFile.seek(0)
        sequences = list(
            SeqIO.parse(seqFile, "fasta")
        )  # Not in same order, but does it matter?
        msa = MultipleSeqAlignment(sequences)
    except:
        shutil.rmtree("tmp/")
        raise
    shutil.rmtree("tmp/")  # rm -rf tmp dir
    return msa

In [61]:
def get_fasta_seq(data):
    return "\n".join(
        [
            SeqRecord(
                Seq(row["sequence"]),
                # id=f"{row['organism'].split()[0]}|{row['accession']}|{row['variant']}",
                id=f"{row['variant'].replace('(', '').replace(')', '')}_{row['organism'].replace(' ', '_')}_{row['accession']}",
                name=row["accession"],
                description=f"organism={row['organism']} phylum={row['phylum']} class={row['class']}",
            ).format("fasta")
            for i, row in data.iterrows()
        ]
    )

In [6]:
def delete_gff_files(directory):
    # Находим все файлы с расширением .gff в указанной папке
    gff_files = glob.glob(os.path.join(directory, "*.gff"))

    # Удаляем каждый файл
    for file_path in gff_files:
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Не удалось удалить файл {file_path}: {e}")

In [45]:
def delete_tree_files(directory):
    # Находим все файлы с расширением .gff в указанной папке
    gff_files = glob.glob(os.path.join(directory, "*.tree"))

    # Удаляем каждый файл
    for file_path in gff_files:
        try:
            os.remove(file_path)
        except Exception as e:
            print(f"Не удалось удалить файл {file_path}: {e}")

In [7]:
class _IdHandler:
    """Generate IDs for GFF3 Parent/Child relationships where they don't exist."""

    def __init__(self):
        self._prefix = "biopygen"
        self._counter = 1
        self._seen_ids = []

    def _generate_id(self, quals):
        """Generate a unique ID not present in our existing IDs."""
        gen_id = self._get_standard_id(quals)
        if gen_id is None:
            while 1:
                gen_id = "%s%s" % (self._prefix, self._counter)
                if gen_id not in self._seen_ids:
                    break
                self._counter += 1
        return gen_id

    def _get_standard_id(self, quals):
        """Retrieve standardized IDs from other sources like NCBI GenBank.

        This tries to find IDs from known key/values when stored differently
        than GFF3 specifications.
        """
        possible_keys = ["transcript_id", "protein_id"]
        for test_key in possible_keys:
            if test_key in quals:
                cur_id = quals[test_key]
                if isinstance(cur_id, tuple) or isinstance(cur_id, list):
                    return cur_id[0]
                else:
                    return cur_id
        return None

    def update_quals(self, quals, has_children):
        """Update a set of qualifiers, adding an ID if necessary."""
        cur_id = quals.get("ID", None)
        # if we have an ID, record it
        if cur_id:
            if not isinstance(cur_id, list) and not isinstance(cur_id, tuple):
                cur_id = [cur_id]
            for add_id in cur_id:
                self._seen_ids.append(add_id)
        # if we need one and don't have it, create a new one
        elif has_children:
            new_id = self._generate_id(quals)
            self._seen_ids.append(new_id)
            quals["ID"] = [new_id]
        return quals

In [8]:
class GFF3Writer:
    """Write GFF3 files starting with standard Biopython objects."""

    def __init__(self):
        pass

    def write(self, recs, out_handle, include_fasta=False, filterids=[]):
        """Write the provided records to the given handle in GFF3 format."""
        self.filterids = filterids
        id_handler = _IdHandler()
        self._write_header(out_handle)
        fasta_recs = []
        try:
            recs = iter(recs)
        except TypeError:
            recs = [recs]
        for rec in recs:
            self._write_rec(rec, out_handle)
            self._write_annotations(rec.annotations, rec.id, len(rec.seq), out_handle)
            for sf in rec.features:
                if sf.id in self.filterids:
                    continue
                sf = self._clean_feature(sf)
                id_handler = self._write_feature(sf, rec.id, out_handle, id_handler)
            if include_fasta and len(rec.seq) > 0:
                fasta_recs.append(rec)
        if len(fasta_recs) > 0:
            self._write_fasta(fasta_recs, out_handle)

    def _clean_feature(self, feature):
        quals = {}
        for key, val in feature.qualifiers.items():
            if not isinstance(val, (list, tuple)):
                val = [val]
            val = [str(x) for x in val]
            quals[key] = val
        feature.qualifiers = quals
        # Support for Biopython 1.68 and above, which removed sub_features
        if not hasattr(feature, "sub_features"):
            feature.sub_features = []
        clean_sub = [self._clean_feature(f) for f in feature.sub_features]
        feature.sub_features = clean_sub
        return feature

    def _write_rec(self, rec, out_handle):
        # if we have a SeqRecord, write out optional directive
        if len(rec.seq) > 0:
            out_handle.write("##sequence-region %s 1 %s\n" % (rec.id, len(rec.seq)))

    def _get_phase(self, feature):
        if "phase" in feature.qualifiers:
            phase = feature.qualifiers["phase"][0]
        elif feature.type == "CDS":
            phase = int(feature.qualifiers.get("codon_start", [1])[0]) - 1
        else:
            phase = "."
        return str(phase)

    def _write_feature(self, feature, rec_id, out_handle, id_handler, parent_id=None):
        """Write a feature with location information."""
        if feature.strand == 1:
            strand = "+"
        elif feature.strand == -1:
            strand = "-"
        else:
            strand = "."
        # remove any standard features from the qualifiers
        quals = feature.qualifiers.copy()
        for std_qual in ["source", "score", "phase"]:
            if std_qual in quals and len(quals[std_qual]) == 1:
                del quals[std_qual]
        # add a link to a parent identifier if it exists
        if parent_id:
            if not "Parent" in quals:
                quals["Parent"] = []
            quals["Parent"].append(parent_id)
        quals = id_handler.update_quals(quals, len(feature.sub_features) > 0)
        if feature.type:
            ftype = feature.type
        else:
            ftype = "sequence_feature"
        parts = [
            str(rec_id),
            feature.qualifiers.get("source", ["feature"])[0],
            ftype,
            str(feature.location.nofuzzy_start + 1),  # 1-based indexing
            str(feature.location.nofuzzy_end),
            feature.qualifiers.get("score", ["."])[0],
            strand,
            self._get_phase(feature),
            self._format_keyvals(quals),
        ]
        out_handle.write("\t".join(parts) + "\n")
        for sub_feature in feature.sub_features:
            id_handler = self._write_feature(
                sub_feature, rec_id, out_handle, id_handler, quals["ID"][0]
            )
        return id_handler

    def _format_keyvals(self, keyvals):
        format_kvs = []
        for key in sorted(keyvals.keys()):
            values = keyvals[key]
            key = key.strip()
            format_vals = []
            if not isinstance(values, list) or isinstance(values, tuple):
                values = [values]
            for val in values:
                # val = urllib.parse.quote(str(val).strip(), safe=":/ ")
                if (key and val) and val not in format_vals:
                    format_vals.append(val)
            format_kvs.append("%s=%s" % (key, ",".join(format_vals)))
        return ";".join(format_kvs)

    def _write_annotations(self, anns, rec_id, size, out_handle):
        """Add annotations which refer to an entire sequence."""
        format_anns = self._format_keyvals(anns)
        if format_anns:
            parts = [
                rec_id,
                "annotation",
                "remark",
                "1",
                str(size if size > 1 else 1),
                ".",
                ".",
                ".",
                format_anns,
            ]
            out_handle.write("\t".join(parts) + "\n")

    def _write_header(self, out_handle):
        """Write out standard header directives."""
        out_handle.write("##gff-version 3\n")

    def _write_fasta(self, recs, out_handle):
        """Write sequence records using the ##FASTA directive."""
        out_handle.write("##FASTA\n")
        SeqIO.write(recs, out_handle, "fasta")

In [9]:
def GFFwrite(recs, out_handle, include_fasta=False, filterids=[]):
    """High level interface to write GFF3 files from SeqRecords and SeqFeatures.

    If include_fasta is True, the GFF3 file will include sequence information
    using the ##FASTA directive.
    """
    writer = GFF3Writer()
    return writer.write(recs, out_handle, include_fasta, filterids)

In [54]:
def generate_draft_seeds(hist_tree, sequence_data, save_directory):
    # generate draft seeds - needs debugging
    draft_seeds_msa = {}

    # create directory if not exist or rewrite the directory
    try:
        os.makedirs(save_directory)
    except FileExistsError:
        # directory already exists
        shutil.rmtree(save_directory)
        os.makedirs(save_directory)

    # hist_tree traversal
    for node in hist_tree.traverse("postorder"):
        print("Processing ", node.name)
        if node.is_root():
            continue
        if node.is_leaf():  # we get sequences for that variant and align them.
            draft_seeds_msa[node.name] = muscle_aln(
                get_fasta_seq(
                    sequence_data.query(
                        f'type=="{node.name}" | variant=="{node.name}"',
                        engine="python",
                    )
                )
            )
            print(node.name, "Alignment length:", len(draft_seeds_msa[node.name]))
            with open(
                f"{save_directory}/{node.name.replace('(', '').replace(')', '')}.fasta",
                "w",
            ) as f:
                f.write(format(draft_seeds_msa[node.name], "fasta"))
        elif not node.is_root():  # we will do profile to profile alignment
            # we should first check if there are seqs with this subvariant as the most specific one
            print(f"\t Node is internal, progressive alignment:")
            msa = muscle_aln(
                get_fasta_seq(sequence_data.query(f'variant=="{node.name}"'))
            )
            draft_seeds_msa[node.name + "_only"] = msa
            print(f"\t\t For {node.name} aligned {len(msa)} sequences")
            # progressively align
            for ch in node.get_children():
                if len(msa) == 0:
                    msa = draft_seeds_msa[ch.name]
                    print(
                        f"\t\t Adding child {node.name} aligned {len(draft_seeds_msa[ch.name])} sequences"
                    )
                    continue
                elif len(draft_seeds_msa[ch.name]) != 0:
                    msa = muscle_p2p_aln(msa, draft_seeds_msa[ch.name])
                    print(
                        f"\t\t Adding child {node.name} aligned {len(draft_seeds_msa[ch.name])} sequences"
                    )
                else:
                    continue
            draft_seeds_msa[node.name] = msa
            print(node.name, "Alignment length:", len(draft_seeds_msa[node.name]))
            with open(
                f"{save_directory}/{node.name.replace('(', '').replace(')', '')}.fasta",
                "w",
            ) as f:
                f.write(format(draft_seeds_msa[node.name], "fasta"))
            with open(
                f"{save_directory}/{node.name.replace('(', '').replace(')', '')}_only.fasta",
                "w",
            ) as f:
                f.write(format(draft_seeds_msa[node.name + "_only"], "fasta"))
        #         print(f"\t\t Final for {node.name} aligned {len(draft_seeds_msa[node.name])} sequences")

In [11]:
# Load curated seeds by traversing the tree
def generate_gff(curated_seeds_msa_form_files, save_directory):
    # firstly, delete all gff files
    # delete_gff_files(save_directory)
    for k, v in curated_seeds_msa_form_files.items():
        if not isinstance(v, list):
            cons = SummaryInfo(v).dumb_consensus(threshold=0.1, ambiguous="X")
            f = hist_features(cons)
            r = SeqRecord(seq=cons, id="Consensus", name="Consensus", features=f)
            with open(
                f"{save_directory}/{k.replace('(', '').replace(')', '')}.gff3", "w"
            ) as j:
                GFFwrite([r], j, filterids=["inner_core"])

In [29]:
def generate_core_seeds(curated_seeds_msa_form_files, save_directory):
    # create directory if not exist or rewrite the directory
    try:
        os.makedirs(save_directory)
    except FileExistsError:
        # directory already exists
        shutil.rmtree(save_directory)
        os.makedirs(save_directory)

    for k, msa in curated_seeds_msa_form_files.items():
        if not isinstance(msa, list):
            core = hist_features(
                SummaryInfo(msa).dumb_consensus(threshold=0.1, ambiguous="X")
            )[1]
            msa_core = msa[:, core.location.start : core.location.end]
            with open(
                f"{save_directory}/{k.replace('(', '').replace(')', '')}.fasta", "w"
            ) as f:
                f.write(msa_core.format("fasta"))

In [12]:
DIR = "/home/l_singh/_scratch/hdb/project_dir/histonedb/CURATED_SET"

In [48]:
with open(f"{DIR}/classification.json") as json_file:
    data = json.load(json_file)
hist_tree = Tree()
dict2tree(hist_tree, data["tree"])
print(hist_tree.get_ascii(show_internal=True))


   /-Archaeal
  |
  |   /-cH1
  |  |
  |  |--generic_H1
  |  |
  |  |-H1.0-H1.0_(Homo_sapiens)
  |  |
  |  |-H1.1-H1.1_(Homo_sapiens)
  |  |
  |  |-H1.10-H1.10_(Homo_sapiens)
  |  |
  |  |-H1.2-H1.2_(Homo_sapiens)
  |  |
  |  |-H1.3-H1.3_(Homo_sapiens)
  |-H1
  |  |-H1.4-H1.4_(Homo_sapiens)
  |  |
  |  |-H1.5-H1.5_(Homo_sapiens)
  |  |
  |  |-OO_H1.8-H1.8_(Homo_sapiens)
  |  |
  |  |--scH1
  |  |
  |  |-TS_H1.6-H1.6_(Homo_sapiens)
  |  |
  |  |-TS_H1.7-H1.7_(Homo_sapiens)
  |  |
  |   \-TS_H1.9
  |
  |                                                                        /-cH2A.10_(Homo_sapiens)
  |                                                                       |
  |                                                                       |--cH2A.11_(Homo_sapiens)
  |                                                                       |
  |                                                                       |--cH2A.1_(Homo_sapiens)
  |                                         

In [49]:
sequence_df = pd.read_csv(f"{DIR}/histones.csv").fillna("")
sequence_df.index = list(sequence_df["accession"])

In [62]:
generate_draft_seeds(
    hist_tree,
    sequence_df,
    f"{DIR}/draft_seeds",
)

Processing  Archaeal
Archaeal Alignment length: 0
Processing  cH1
cH1 Alignment length: 0
Processing  generic_H1
generic_H1 Alignment length: 13
Processing  H1.0_(Homo_sapiens)
H1.0_(Homo_sapiens) Alignment length: 1
Processing  H1.0
	 Node is internal, progressive alignment:
		 For H1.0 aligned 14 sequences
		 Adding child H1.0 aligned 1 sequences
H1.0 Alignment length: 15
Processing  H1.1_(Homo_sapiens)
H1.1_(Homo_sapiens) Alignment length: 1
Processing  H1.1
	 Node is internal, progressive alignment:
		 For H1.1 aligned 0 sequences
		 Adding child H1.1 aligned 1 sequences
H1.1 Alignment length: 1
Processing  H1.10_(Homo_sapiens)
H1.10_(Homo_sapiens) Alignment length: 1
Processing  H1.10
	 Node is internal, progressive alignment:
		 For H1.10 aligned 5 sequences
		 Adding child H1.10 aligned 1 sequences
H1.10 Alignment length: 6
Processing  H1.2_(Homo_sapiens)
H1.2_(Homo_sapiens) Alignment length: 1
Processing  H1.2
	 Node is internal, progressive alignment:
		 For H1.2 aligned 0 seq

In [63]:
curated_seeds_msa_form_files = {}
for node in hist_tree.traverse("postorder"):
    if os.path.isfile(
        f"{DIR}/draft_seeds/{node.name.replace('(', '').replace(')', '')}.fasta"
    ):
        print("Loading ", node.name)
        curated_seeds_msa_form_files[node.name] = (
            AlignIO.read(
                f"{DIR}/draft_seeds/{node.name.replace('(', '').replace(')', '')}.fasta",
                "fasta",
            )
            if os.path.getsize(
                f"{DIR}/draft_seeds/{node.name.replace('(', '').replace(')', '')}.fasta"
            )
            > 0
            else []
        )

Loading  Archaeal
Loading  cH1
Loading  generic_H1
Loading  H1.0_(Homo_sapiens)
Loading  H1.0
Loading  H1.1_(Homo_sapiens)
Loading  H1.1
Loading  H1.10_(Homo_sapiens)
Loading  H1.10
Loading  H1.2_(Homo_sapiens)
Loading  H1.2
Loading  H1.3_(Homo_sapiens)
Loading  H1.3
Loading  H1.4_(Homo_sapiens)
Loading  H1.4
Loading  H1.5_(Homo_sapiens)
Loading  H1.5
Loading  H1.8_(Homo_sapiens)
Loading  OO_H1.8
Loading  scH1
Loading  H1.6_(Homo_sapiens)
Loading  TS_H1.6
Loading  H1.7_(Homo_sapiens)
Loading  TS_H1.7
Loading  TS_H1.9
Loading  H1
Loading  cH2A.10_(Homo_sapiens)
Loading  cH2A.11_(Homo_sapiens)
Loading  cH2A.1_(Homo_sapiens)
Loading  cH2A.2_(Homo_sapiens)
Loading  cH2A.3_(Homo_sapiens)
Loading  cH2A.4_(Homo_sapiens)
Loading  cH2A.5_(Homo_sapiens)
Loading  cH2A.6_(Homo_sapiens)
Loading  cH2A.7_(Homo_sapiens)
Loading  cH2A.8_(Homo_sapiens)
Loading  cH2A.9_(Homo_sapiens)
Loading  cH2A_(Homo_sapiens)
Loading  cH2A.1_(Mus_musculus)
Loading  cH2A_(Mus_musculus)
Loading  cH2A_(Mammalia)
Loading 

In [64]:
generate_gff(curated_seeds_msa_form_files, f"{DIR}/draft_seeds")

In [65]:
generate_core_seeds(curated_seeds_msa_form_files, f"{DIR}/draft_seeds_core")

In [66]:
def run_bash_command(command_string):
    try:
        # Запускаем команду bash из строки
        result = subprocess.run(
            ["bash", "-c", command_string], check=True, text=True, capture_output=True
        )
    except subprocess.CalledProcessError as e:
        print("Ошибка выполнения скрипта:", e.stderr)

In [42]:
shutil.rmtree("tmp/")

In [67]:
for node in hist_tree.traverse("postorder"):
    if os.path.isfile(
        f"{DIR}/draft_seeds/{node.name.replace('(', '').replace(')', '')}.fasta"
    ) and os.path.isfile(
        f"{DIR}/draft_seeds_core/{node.name.replace('(', '').replace(')', '')}.fasta"
    ):
        print("Building ", node.name)
        name = node.name.replace("(", "").replace(")", "")
        os.makedirs("tmp/")
        run_bash_command(
            f"FastTree -nosupport < {DIR}/draft_seeds_core/{name}.fasta > tmp/tree 2> /dev/null"
        )
        run_bash_command(
            f"sed '$ s/.$/Consensus;/' tmp/tree > {DIR}/draft_seeds/{name}.tree"
        )
        shutil.rmtree("tmp/")  # rm -rf tmp dir

Building  generic_H1
Building  H1.0_(Homo_sapiens)
Building  H1.0
Building  H1.1_(Homo_sapiens)
Building  H1.1
Building  H1.10_(Homo_sapiens)
Building  H1.10
Building  H1.2_(Homo_sapiens)
Building  H1.2
Building  H1.3_(Homo_sapiens)
Building  H1.3
Building  H1.4_(Homo_sapiens)
Building  H1.4
Building  H1.5_(Homo_sapiens)
Building  H1.5
Building  H1.8_(Homo_sapiens)
Building  OO_H1.8
Building  scH1
Building  H1.6_(Homo_sapiens)
Building  TS_H1.6
Building  H1.7_(Homo_sapiens)
Building  TS_H1.7
Building  TS_H1.9
Building  H1
Building  cH2A.10_(Homo_sapiens)
Building  cH2A.11_(Homo_sapiens)
Building  cH2A.1_(Homo_sapiens)
Building  cH2A.2_(Homo_sapiens)
Building  cH2A.3_(Homo_sapiens)
Building  cH2A.4_(Homo_sapiens)
Building  cH2A.5_(Homo_sapiens)
Building  cH2A.6_(Homo_sapiens)
Building  cH2A.7_(Homo_sapiens)
Building  cH2A.8_(Homo_sapiens)
Building  cH2A.9_(Homo_sapiens)
Building  cH2A_(Homo_sapiens)
Building  cH2A.1_(Mus_musculus)
Building  cH2A_(Mus_musculus)
Building  cH2A_(Mammalia)
